# Test #5 - Complete example


## Nom à trouver

In this test, pyKasso is used on a simple rectangular domain, with one inlet in the center of the upper left limit of the model, and one outlet in the center of the bottom right limit of the model.
pyKasso is firstly used with the isotropic fast-marching algorithm then with the anisotropic one.  

In [1]:
# Loads pyKasso and other useful packages
import numpy as np
import pykasso as pk
import pykasso.tools as pkt
import pykasso.visualization as pkv

In [2]:
# Creates a new project directory
pk.create_project('geometry_05')
path_project = 'geometry_05/outputs/project_state.yaml'

CAUTION: You are using the development version of this package.


In [3]:
# Constructs the settings
sks_settings = {
    'sks' : {
        'seed' : 5555, 
    },
    'grid' : {
        'x0' : 0,
        'y0' : 0,
        'z0' : 100,
        'nx' : 100,
        'ny' : 100,
        'nz' : 50,
        'dx' : 10,
        'dy' : 10,
        'dz' : 10
    },
    'domain' : {},
    'outlets' : {
        'number'     : 1,
        'data'       : [[994, 500]],
        'importance' : [1],
        'mode'       : 'domain_bottom',
    },
    'inlets' : {
        'number'     : 2,
        'data'       : [[0, 250], [0, 994]],
        'per_outlet' : [1],
        'importance' : [1],
        'mode'       : 'domain_surface',
    },
}

In [4]:
# Constructs a grid it will help construct the other arrays
grid = pk.Grid(**sks_settings['grid'])
x = grid.X[:,:,0]
y = grid.Y[:,:,0]

# Constructs the topography
topography = np.sin(y/120)*80 + 500

# Constructs the water level elevation
water_level_elevation = np.linspace(400, 300, 100)
water_level_elevation = np.repeat(water_level_elevation[:, np.newaxis], 100, axis=1)
water_level_elevation = np.sin(x/40)*7 + water_level_elevation

# Constructs the bedrock
z_valley = np.linspace(300, 100, 100)
bedrock = []
for z in z_valley:
    line = np.append(np.linspace(200+z, z, 50), np.linspace(z, 200+z, 50)) 
    bedrock.append(line)
bedrock = np.vstack(bedrock)

# Visualizes the arrays
# pkv.show_array(topography)
# pkv.show_array(water_level_elevation)
# pkv.show_array(bedrock)

# Updates the settings
sks_settings['domain']['topography']  = topography
sks_settings['domain']['water_level'] = water_level_elevation
sks_settings['domain']['bedrock']     = bedrock

In [5]:
for algorithm in ['Isotropic3', 'Riemann3']:
    sks_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=sks_settings)
    sim.build_model()
    sim.compute_karst_network()

In [7]:
visualization = pkv.Visualization(path_project)
settings = {
    'ghosts'  : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'surfaces' : ['topography', 'water_level', 'bedrock']
}
visualization.show([0,1], ['karst'], settings=settings)